In [204]:
import os
from citrination_client import CitrinationClient
from citrination_client import PifSystemReturningQuery, PifSystemQuery
from citrination_client import DataQuery, DatasetQuery, ChemicalFieldQuery, ChemicalFilter, Filter
from citrination_client import PropertyQuery, FieldQuery, ProcessStepQuery, ValueQuery, IdQuery
from pypif.pif import dumps
import pandas as pd
from textwrap import dedent
from string_misc import indent_block

In [3]:
client = CitrinationClient(os.environ['CITRINATION_API_KEY'],'https://citrination.com')

In [17]:
config = os.path.join(os.environ['HOME'],'OneDrive - Colorado School of Mines/Research/MIDDMI/TCO/Data/config')
procdetdf = pd.read_excel(os.path.join(config,'procdets.xlsx'))
procdetdf

,ProcStep,ProcStepDetail,ExtractAs
0,Pulsed Laser Deposition,Targets used,targets
1,Pulsed Laser Deposition,Laser Energy,laser_energy
2,Pulsed Laser Deposition,Laser Voltage,laser_voltage
3,Pulsed Laser Deposition,Temperature,temp
4,Pulsed Laser Deposition,Temperature Gradient,temp_grad
5,Pulsed Laser Deposition,BZC19 Pulses/Cycle,bzc19_ppc
6,Pulsed Laser Deposition,BZC91 Pulses/Cycle,bzc91_ppc
7,Pulsed Laser Deposition,BZF19 Pulses/Cycle,bzf19_ppc
8,Pulsed Laser Deposition,BZY82 Pulses/Cycle,bzy82_ppc
9,Pulsed Laser Deposition,BZYCF1135 Pulses/Cycle,bzycf1135_ppc


In [6]:
def create_query(datasetid, props, procdetdf,size=None):

    procsteps = list(procdetdf.ProcStep.unique())

    propquery = []
    procquery = []
    #store query string for validation
    propquerytxtlist = []
    procquerytxtlist = []

    for prop, exas in props.items():
        pq = PropertyQuery(
                name=FieldQuery(
                    filter=[Filter(equal=prop)]),
                value=FieldQuery(
                    extract_as=exas),
                logic="MUST"
            )
        pqt = dedent('''\
                PropertyQuery(
                    name=FieldQuery(
                        filter=[Filter(equal="{}")]),
                    value=FieldQuery(
                        extract_as="{}"),
                    logic="MUST"
                )'''.format(prop,exas))
        propquery.append(pq)
        propquerytxtlist.append(pqt)

    for procstep in procsteps:
        detquery = []
        detquerytxtlist = []
        pdf = procdetdf[procdetdf.ProcStep==procstep]
        for det, exas in zip(pdf.ProcStepDetail, pdf.ExtractAs):
            dq = ValueQuery(
                    name=FieldQuery(
                        filter=[Filter(equal=det)]),
                    value=FieldQuery(
                        extract_as=exas),
                    logic="MUST"
                )
            dqt = dedent('''\
                    ValueQuery(
                        name=FieldQuery(
                            filter=[Filter(equal="{}")]),
                        value=FieldQuery(
                            extract_as="{}"),
                        logic="MUST"
                    )'''.format(det,exas))
            detquery.append(dq)
            detquerytxtlist.append(dqt)

        pq = ProcessStepQuery(
                name=FieldQuery(
                    filter=[Filter(equal=procstep)]),
                details=detquery
            )
        detquerytxt = indent_block(',\n'.join(detquerytxtlist), level=6)
        pcqt = dedent('''\
                ProcessStepQuery(
                    name=FieldQuery(
                        filter=[Filter(equal="{}")]),
                    details=[\n{}
                    ]
                )'''.format(procstep,detquerytxt))
        procquery.append(pq)
        procquerytxtlist.append(pcqt)

    pifsysquery = PifSystemQuery(
                        properties = propquery,
                        preparation = procquery,
                        chemical_formula = ChemicalFieldQuery(
                                extract_as="formula")
                    )

    propquerytxt = indent_block(',\n'.join(propquerytxtlist), level=6)
    procquerytxt = indent_block(',\n'.join(procquerytxtlist), level=6)
    pifsysquerytxt = indent_block(dedent('''\
                    properties = [\n{}
                    ],
                    preparation = [\n{}
                    ],
                    chemical_formula = ChemicalFieldQuery(
                            extract_as="formula")'''.format(propquerytxt,procquerytxt)), level=7)


    query = PifSystemReturningQuery(
                size = size,
                query = DataQuery(
                            dataset=DatasetQuery(
                                id=[Filter(equal=str(dataset_id))]
                            ),
                            system = pifsysquery
                )
            )
    querytxt = dedent('''\
                PifSystemReturningQuery(
                    size = {},
                    query = DataQuery(
                                dataset=DatasetQuery(
                                    id=[Filter(equal={})]
                                ),
                                system = PifSystemQuery(\n{}
                                )
                    )
                )'''.format(size,dataset_id,pifsysquerytxt))

    return query, querytxt

In [7]:
dataset_id = 155268
props = {'FWHM of 110 peak':'fwhm_110'}

query, querytxt = create_query(dataset_id,props,procdetdf,size=2)
query_result = client.search(query)
print("{} total hits\n".format(query_result.total_num_hits))
for i in range(len(query_result.hits)):
    print(dumps(query_result.hits[i].extracted,indent=2))

1295 total hits

{
  "bzyf631_ppc": "0",
  "temp": "850",
  "avg_ppc": "191.0",
  "fwhm_110": "0.74988985",
  "bzc91_ppc": "0",
  "pressure": "50.0",
  "laser_voltage": "22.5",
  "targets": "BZF19",
  "bzc19_ppc": "0",
  "bzy82_ppc": "0",
  "bzf19_ppc": "191",
  "temp_grad": "N",
  "bzycf1135_ppc": "0",
  "num_cyc": "200.0",
  "laser_freq": "10",
  "b12zy82_ppc": "0",
  "laser_energy": "400.0",
  "formula": "Ba0.96873Co0.0Fe0.84248Y0.0Zr0.15752O3",
  "substrate": "SiO2"
}
{
  "bzyf631_ppc": "24",
  "temp": "850",
  "avg_ppc": "28.666666666666668",
  "fwhm_110": "0.53148776",
  "bzc91_ppc": "25",
  "pressure": "50.0",
  "laser_voltage": "18.5",
  "targets": "BZC91/ BZYF631/ BZF19",
  "bzy82_ppc": "0",
  "bzc19_ppc": "0",
  "bzf19_ppc": "37",
  "temp_grad": "N",
  "bzycf1135_ppc": "0",
  "num_cyc": "347.0",
  "laser_freq": "10",
  "b12zy82_ppc": "0",
  "laser_energy": "600.0",
  "formula": "Ba0.84821Co0.00789Fe0.3338Y0.10492Zr0.55339O3",
  "substrate": "SiO2"
}


In [8]:
print(querytxt)

PifSystemReturningQuery(
    size = 2,
    query = DataQuery(
                dataset=DatasetQuery(
                    id=[Filter(equal=155268)]
                ),
                system = PifSystemQuery(
            properties = [
                PropertyQuery(
                    name=FieldQuery(
                        filter=[Filter(equal="FWHM of 110 peak")]),
                    value=FieldQuery(
                        extract_as="fwhm_110",
                        logic="MUST")
                )
            ],
            preparation = [
                ProcessStepQuery(
                    name=FieldQuery(
                        filter=[Filter(equal="Pulsed Laser Deposition")]),
                    details=[
                        PropertyQuery(
                            name=FieldQuery(
                                filter=[Filter(equal="Targets used")]),
                            value=FieldQuery(
                                extract_as="targets",
               

In [23]:
len(query_result.hits)

2

In [16]:
query = PifSystemReturningQuery(
    size = 2,
    query = DataQuery(
                dataset=DatasetQuery(
                    id=[Filter(equal=155268)]
                ),
                system = PifSystemQuery(
            properties = [
                PropertyQuery(
                    name=FieldQuery(
                        filter=[Filter(equal="FWHM of 110 peak")]),
                    value=FieldQuery(
                        extract_as="fwhm_110",
                        logic="MUST")
                )
            ],
            preparation = [
                ProcessStepQuery(
                    name=FieldQuery(
                        filter=[Filter(equal="Pulsed Laser Deposition")]),
                    details=[
                        ValueQuery(
                            name=FieldQuery(
                                filter=[Filter(equal="Targets used")]),
                            value=FieldQuery(
                                extract_as="targets"),
                            logic="MUST"
                        ),
                        ValueQuery(
                            name=FieldQuery(
                                filter=[Filter(equal="X")]),
                            value=FieldQuery(
                                extract_as="substrate"),
                            logic="MUST"
                        )
                    ]
                )
            ],
            chemical_formula = ChemicalFieldQuery(
                    extract_as="formula")
                )
    )
)
                    

query_result = client.search(query)
query_result.total_num_hits

0

In [12]:
prop_name_query = FieldQuery(filter=[Filter(equal="FWHM of 110 peak")])
prop_value_query = FieldQuery(extract_as="FWHM of 110 peak", extract_all=True)
prop_query = PropertyQuery(name=prop_name_query, value=prop_value_query)
formula_query = ChemicalFieldQuery(extract_as="formula")
process_detail_names = ["Targets used", "Laser Energy", "Laser Voltage", "Temperature", 
                        "Temperature Gradient", "BZC19 Pulses/Cycle", "BZC91 Pulses/Cycle",
                       "BZF19 Pulses/Cycle", "BZY82 Pulses/Cycle", "BZYCF1135 Pulses/Cycle",
                       "BZYF631 Pulses/Cycle", "B12ZY82 Pulses/Cycle", "Average Target Pulses/Cycle",
                       "Number of Cycles", "Laser Pulse Frequency", "Deposition Pressure", "Substrate"]
process_detail_queries = []
for detail_name in process_detail_names:
    det_name_query = FieldQuery(filter=[Filter(equal=detail_name)])
    value_query = FieldQuery(extract_as=detail_name)
    pro_det_query = ValueQuery(name=det_name_query, value=value_query, logic="MUST")
    process_detail_queries.append(pro_det_query)
process_name_query = FieldQuery(filter=[Filter(equal="Pulsed Laser Deposition")])
process_step_query = ProcessStepQuery(name=process_name_query, details=process_detail_queries)
system_query = PifSystemQuery(chemical_formula=formula_query, properties=prop_query, preparation=process_step_query)
dataset_query = DatasetQuery(id=[Filter(equal='155268')])
data_query = DataQuery(dataset=dataset_query, system=system_query)
test_query = PifSystemReturningQuery(size=100, query=data_query)
pif_search_result = client.search(test_query)

pif_search_result.total_num_hits

1292

In [11]:
view_id = '4198'
inputs = [{"Chemical formula":'Ba0.88873Co0.33364Fe0.26906Y0.08999Zr0.30731O3',"A_avg_X":0.89}]
resp = client.predict(view_id,inputs)

In [12]:
resp

{'candidates': [{}]}

In [31]:
query = PifSystemReturningQuery(
    size=10,
    query=DataQuery(
        dataset=DatasetQuery(
            id=Filter(equal='155268')
        ),
        system=PifSystemQuery(
            chemical_formula=ChemicalFieldQuery(
                filter=ChemicalFilter(
                    equal='Ba#Co#Fe#Y#Zr#O3',
                    partial=True
                ),
                extract_as="formula"
            )
        )
    )
)

query_result = client.search(query)
query_result.total_num_hits

0

In [22]:
for i in range(len(query_result.hits)):
    print(dumps(query_result.hits[i].extracted,indent=2))

{
  "formula": "Ba0.81429Co0.03986Fe0.0Y0.0Zr0.96014O3"
}
{
  "formula": "Ba0.96873Co0.0Fe0.84248Y0.0Zr0.15752O3"
}
{
  "formula": "Ba0.84821Co0.00789Fe0.3338Y0.10492Zr0.55339O3"
}
{
  "formula": "Ba0.69726Co0.00087Fe0.0Y0.15574Zr0.8434O3"
}
{
  "formula": "Ba0.91509Co0.1641Fe0.34717Y0.05624Zr0.43249O3"
}
{
  "formula": "Ba0.88055Co0.0Fe0.41686Y0.15973Zr0.42341O3"
}
{
  "formula": "Ba0.98342Co0.59811Fe0.19831Y0.04896Zr0.15462O3"
}
{
  "formula": "Ba0.90443Co0.0Fe0.43014Y0.14697Zr0.4229O3"
}
{
  "formula": "Ba0.9407Co0.25606Fe0.39704Y0.13925Zr0.20765O3"
}
{
  "formula": "Ba0.86915Co0.43614Fe0.45293Y0.0Zr0.11094O3"
}


In [207]:
query = PifSystemReturningQuery(
    size=10,
    query=DataQuery(
        dataset=DatasetQuery(
            id=Filter(equal='155268')
        ),
        system=PifSystemQuery(
            chemical_formula=ChemicalFieldQuery(
                extract_as="formula"
            ),
            preparation = [
                ProcessStepQuery(
                    name=FieldQuery(
                        filter=[Filter(equal="Pulsed Laser Deposition")]),
                    details=[
                        ValueQuery(
                            name=FieldQuery(
                                filter=[Filter(equal="Laser Energy")]),
                            value=FieldQuery(
                                extract_as="laser_energy",
                                filter=[Filter(max="500")]
                            ),
                            logic="MUST"
                        )
                    ]
                )
            ],
            properties = PropertyQuery(
                name=FieldQuery(
                    filter=[Filter(equal='FWHM of 110 peak')]),
                value=FieldQuery(extract_as="fhwm_110")
            ),
            ids = IdQuery(
                name=FieldQuery(
                    filter=[Filter(equal='Sample number')]),
                value=FieldQuery(
                    extract_as="sample")
            )
        )
    )
)

query_result = client.search(query)
query_result.total_num_hits
for i in range(len(query_result.hits)):
    print(dumps(query_result.hits[i].extracted,indent=2))

{
  "fhwm_110": "0.74988985",
  "laser_energy": "400.0",
  "formula": "Ba0.96873Co0.0Fe0.84248Y0.0Zr0.15752O3",
  "sample": "01139"
}
{
  "fhwm_110": "0.50759244",
  "laser_energy": "500.0",
  "formula": "Ba0.90727Co0.5702Fe0.0Y0.0Zr0.4298O3",
  "sample": "01098"
}
{
  "fhwm_110": "0.45376262",
  "laser_energy": "500.0",
  "formula": "Ba0.95958Co0.50245Fe0.37822Y0.0Zr0.11933O3",
  "sample": "01097"
}
{
  "fhwm_110": "0.4184168",
  "laser_energy": "500.0",
  "formula": "Ba0.96356Co0.49027Fe0.37891Y0.0Zr0.13082O3",
  "sample": "01097"
}
{
  "fhwm_110": "0.55419463",
  "laser_energy": "425.0",
  "formula": "Ba0.83507Co0.02038Fe0.39412Y0.0Zr0.5855O3",
  "sample": "01027"
}
{
  "fhwm_110": "0.41783518",
  "laser_energy": "500.0",
  "formula": "Ba0.96135Co0.55665Fe0.32512Y0.0Zr0.11824O3",
  "sample": "01097"
}
{
  "fhwm_110": "0.43435109",
  "laser_energy": "500.0",
  "formula": "Ba0.90533Co0.26376Fe0.35709Y0.0Zr0.37915O3",
  "sample": "01099"
}
{
  "fhwm_110": "0.44034666",
  "laser_energy"

In [203]:
import citrination_client
dir(citrination_client)

['BaseFieldQuery',
 'BaseObjectQuery',
 'BooleanFilter',
 'ChemicalFieldQuery',
 'ChemicalFilter',
 'CitrinationClient',
 'ClassificationQuery',
 'CompositionQuery',
 'DataQuery',
 'DataScope',
 'DatasetMultiSearchResult',
 'DatasetMultiSearchResultElement',
 'DatasetQuery',
 'DatasetReturningQuery',
 'DatasetSearchHit',
 'DatasetSearchResult',
 'DisplayItemQuery',
 'ExtractionSort',
 'FieldQuery',
 'FileMultiSearchResult',
 'FileMultiSearchResultElement',
 'FileQuery',
 'FileReferenceQuery',
 'FileReturningQuery',
 'FileSearchHit',
 'FileSearchResult',
 'Filter',
 'IdQuery',
 'MultiQuery',
 'NameQuery',
 'PagesQuery',
 'PifMultiSearchResult',
 'PifMultiSearchResultElement',
 'PifSearchHit',
 'PifSearchResult',
 'PifSystemQuery',
 'PifSystemReturningQuery',
 'ProcessStepQuery',
 'PropertyQuery',
 'QuantityQuery',
 'ReferenceQuery',
 'SourceQuery',
 'ValueQuery',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__'

In [206]:
dir(PifSystemQuery)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_convert_to_dictionary',
 '_get_object',
 'as_dictionary',
 'chemical_formula',
 'classifications',
 'composition',
 'extract_all',
 'extract_as',
 'extract_when_missing',
 'ids',
 'length',
 'logic',
 'names',
 'offset',
 'preparation',
 'properties',
 'quantity',
 'query',
 'references',
 'simple',
 'simple_weight',
 'source',
 'sub_systems',
 'tags',
 'uid',
 'updated_at',
 'weight']